# Web API

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


[08:57:29] Created task 'webAPI' with task_id 'fdve-6de74884-3432-4588-9d9e-b00087e17b10v1'.          ]8;id=629041;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=74011;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[08:57:36] status = queued                                                                            ]8;id=749010;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=657195;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#269\269]8;;\

Output()

[08:57:45] status = preprocess                                                                        ]8;id=240736;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=739897;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#263\263]8;;\

...

           status = postprocess                                                                       ]8;id=87017;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=934419;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#330\330]8;;\

Output()

[08:58:00] status = success                                                                           ]8;id=945480;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=432389;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#337\337]8;;\

Output()

[08:58:03] loading SimulationData from data/sim.hdf5                                                  ]8;id=374166;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=594380;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../_autosummary/tidy3d.web.webapi.run.html#tidy3d.web.webapi.run) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../_autosummary/tidy3d.web.webapi.start.html) and then exit the session and load the results at a later time using [web.load](../_autosummary/tidy3d.web.webapi.load.html#tidy3d.web.webapi.load).


## Job Container

The [web.Job](../_autosummary/tidy3d.web.container.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../_autosummary/tidy3d.Simulation.html).

While [Job](../_autosummary/tidy3d.web.container.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call [.run()] so we will do that now.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")


[08:58:03] Created task 'job' with task_id 'fdve-e7202274-e32b-4efe-98b9-e44efe2456a7v1'.             ]8;id=702028;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=907613;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[08:58:06] status = queued                                                                            ]8;id=377331;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=868826;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#269\269]8;;\

Output()

[08:58:08] status = preprocess                                                                        ]8;id=903736;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=344816;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#263\263]8;;\

...

           status = postprocess                                                                       ]8;id=853108;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=79260;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#330\330]8;;\

Output()

[08:58:24] status = success                                                                           ]8;id=186871;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=413260;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#337\337]8;;\

Output()

[08:58:28] loading SimulationData from data/sim.hdf5                                                  ]8;id=65038;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=684154;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

Another convenient thing about [Job](../_autosummary/tidy3d.web.container.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [5]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

[08:58:29] loading SimulationData from data/sim.hdf5                                                  ]8;id=905089;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=431243;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../_autosummary/tidy3d.web.container.Batch.html) object.
The batch object is like a [Job](../_autosummary/tidy3d.web.container.Job.html), but stores task metadata for a series of simulations.

In [6]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


[08:58:29] Created task 'sim_0' with task_id 'fdve-e7352451-fa3c-42b7-8006-8360a3631f3av1'.           ]8;id=557888;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=230503;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[08:58:31] Created task 'sim_1' with task_id 'fdve-27be36ff-f6b7-4bda-9176-5a931d8bb6fav1'.           ]8;id=202384;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=494228;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[08:58:32] Created task 'sim_2' with task_id 'fdve-d049ece8-d640-4956-97b4-fc1546f245ffv1'.           ]8;id=661454;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=214778;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#139\139]8;;\

Output()

[08:58:35] Started working on Batch.                                                               ]8;id=563462;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=448632;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#402\402]8;;\

Output()

[08:58:59] Batch complete.                                                                         ]8;id=372887;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=433024;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#436\436]8;;\

When the batch is completed, the output is not a [SimulationData](../_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../_autosummary/tidy3d.web.container.BatchData.html).  The data within this [BatchData](../_autosummary/tidy3d.web.container.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [7]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[08:59:03] loading SimulationData from data/fdve-e7352451-fa3c-42b7-8006-8360a3631f3av1.hdf5          ]8;id=831463;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=832264;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

Output()

[08:59:05] loading SimulationData from data/fdve-27be36ff-f6b7-4bda-9176-5a931d8bb6fav1.hdf5          ]8;id=758926;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=893696;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

Output()

[08:59:08] loading SimulationData from data/fdve-d049ece8-d640-4956-97b4-fc1546f245ffv1.hdf5          ]8;id=989146;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=262485;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

{'sim_0': 13540217.099674325, 'sim_1': 13540217.099674325, 'sim_2': 13540217.099674325}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../_autosummary/tidy3d.web.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [8]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


Output()

In [9]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[08:59:38] loading SimulationData from ./fdve-a802be6a-507e-4636-b1dc-11628db174b9v1.hdf5             ]8;id=348007;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=283331;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

Output()

[08:59:39] loading SimulationData from ./fdve-35574312-ed35-4da7-92c4-d1552414449dv1.hdf5             ]8;id=731163;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=626334;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

Output()

[08:59:41] loading SimulationData from ./fdve-dedd1b16-76b3-44b2-826c-af2e29ec2073v1.hdf5             ]8;id=183605;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=779406;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#509\509]8;;\

{'sim_0': 13540217.099674325, 'sim_1': 13540217.099674325, 'sim_2': 13540217.099674325}
